In [ ]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
# Carregar base de CMED
base_cmed = pd.read_excel("Lista de Preço CMED.xls", 
                          skiprows=48,
                          decimal=',')

eans = base_cmed['EAN 1'][:30]
#preco = base_cmed[['PF 0%','PF 22%']]

In [ ]:
def farmacia_preco(ean):
    url = 'https://www.urlfarmacia.com.br/'
    ean = ean

    url_final = f'{url}{ean}?_q={ean}&map=ft'
    
    response = requests.get(url_final)
    soup = BeautifulSoup(response.content, 'html.parser')
        
    nome_elemento = soup.find('span', {'class': 'subclass-nome'})
    preco_elemento = soup.find('div', {'class': 'subclass-preco'})
    
    if nome_elemento is None:
        nome = 'Produto não encontrado'
    else:
        nome = nome_elemento.text.strip()
    
    if preco_elemento is None:
        preco = 0
    else:
        preco = preco_elemento.text.strip().removeprefix('R$\xa0')
        preco = float(preco.replace(',', '.'))
    
    data = {
        'NOME_FARMACIA': [nome],
        'PRECO_FARMACIA': [preco],
        'EAN1':[ean]
            }

    df = pd.DataFrame(data)
    return(df)

In [ ]:
precos = pd.DataFrame()

for i in eans:
    preco = farmacia_preco(i)
    precos = pd.concat([precos, preco], axis=0)

In [ ]:
#Carregar base de dados CMED com campos que serão analizados
base_cmed_filtrada = base_cmed[['SUBSTÂNCIA',
                                'CNPJ',
                                'LABORATÓRIO',
                                'REGISTRO',
                                'EAN 1',
                                'PF Sem Impostos', 
                                'PF 0%', 
                                'PF 22%', 
                                'PMC 0%',
                                'PMC 22%'
                               ]]

In [ ]:
# Agrupar tabelas por EAN1
base_comparacao = pd.merge(precos, base_cmed_filtrada, 
                           left_on='EAN1', 
                           right_on='EAN 1', 
                           how='left')

In [ ]:
base_comparacao[['EAN1',
                 'SUBSTÂNCIA',
                 'PRECO_FARMACIA',
                 'PF Sem Impostos',
                 'PF 22%',
                 'PMC 0%',
                 'PMC 22%']]